In [ ]:
import cv2
import numpy as np
import mediapipe as mp



cap = cv2.VideoCapture("Woman With Umbrella Walking.mp4")
#Video path : "C:\\Users\\chinm\\Downloads\\Woman With Umbrella Walking.mp4"


mp_drawings = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a,b,c):
    a = np.array(a) # Each point consist of its x,y,z values. so a[0] = x; a[1] = y; a[2] = z
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]- b[1], c[0]-b[0]) - np.arctan2(a[1]- b[1], a[0]- b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180:
        angle = 360-angle

    return angle 
count = 0
stage = "Down"
#SETUP MEDIAPIPE INSTANCE..
with mp_pose.Pose(min_detection_confidence = 0.6 , min_tracking_confidence = 0.6) as pose:
    while cap.isOpened():
        ret , img0 = cap.read()
        img = cv2.resize(img0 , (964,484))

        image = cv2.cvtColor(img , cv2.COLOR_BGR2RGB) #RECOLOURING THE IMAGE..{Opencv = BGR , Mediapipe = RGB}
        image.flags.writeable = False  #Setting image to unwritable...

        result = pose.process(image)  #MAKES DETECTION..# Storing processed image in result

        image.flags.writeable = True  #Setting image back to writable..
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #Back to BGR

        #Extract LANDMARKS...
        try:
            
            landmarks = result.pose_landmarks.landmark
            #print(landmarks) #Squeezed text..

            
            #Get coordinates..
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            #calculate angle
            #if calculate_angle(shoulder,elbow,wrist)> 70 and calculate_angle(shoulder,elbow,wrist) < 100 :
            #print("Good Job")
            angle =  calculate_angle(shoulder,elbow,wrist)

            cv2.putText(image,str(angle), tuple(np.multiply(elbow,[964,484]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (255,255,255),2,cv2.LINE_AA)    

            if angle > 160 :
                stage = "Down"
            if angle < 30 and stage == "Down":
                stage = "Up"
                count = count + 1
                print(count)
        except:
            pass

        #Rectangle
        cv2.rectangle(image,(0,0) , (275,75) , (245,117,16) , cv2.FILLED)
        cv2.putText(image,"REPS", (5,25),cv2.FONT_HERSHEY_SIMPLEX, 1 , (255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,str(count), (5,60),cv2.FONT_HERSHEY_SIMPLEX, 1 , (255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,"Stage", (120,25),cv2.FONT_HERSHEY_SIMPLEX, 1 , (255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,stage, (120,60),cv2.FONT_HERSHEY_SIMPLEX, 1 , (255,255,255),2,cv2.LINE_AA)
        

    


        
        #Render Detections...
        mp_drawings.draw_landmarks(image , result.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawings.DrawingSpec(color =(245,177,66), thickness = 2 , circle_radius = 2), mp_drawings.DrawingSpec(color =(245,66,230), thickness = 2 , circle_radius = 2)) 
    
  
        #print(result)


        
        cv2.imshow("Video", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    



#print(result.pose_landmarks)
#print(mp_pose.POSE_CONNECTIONS)
#print(len(landmarks))  #Number of points...

#for lndmarks in mp_pose.PoseLandmark:
    #print(lndmarks)





#print(mp_pose.PoseLandmark.RIGHT_WRIST.value) # prints 16
#print(landmarks[mp_pose.PoseL
print(landmarks)